In [40]:
# Imports, always include this at the beginning

# No science without NumPy
import numpy as np

# Of course we want Glotaran
import glotaran as gta

# This things we just need for simulations, you can leave them out for fitting
from lmfit import Parameters

In [46]:
# First we define our model specification. See the documentation for details how to do so.
model_spec = '''
type: kinetic

parameters: [101e-1, 202e-5,505e-6]

compartments: [s1, s2, s3]

megacomplexes:
    - label: mc1
      k_matrices: [k1]

k_matrices:
  - label: "k1"
    matrix: {
      '("s1","s1")': 1,
      '("s2","s2")': 2,
      '("s3","s3")': 3,
}

initial_concentrations: []

irf: []

datasets: 
  - label: dataset1
    type: spectral
    megacomplexes: [mc1]
'''

# Then we can create our model
model = parse_yml(model_spec)

In [47]:
# Now let us define some simulation parameters
sim_params = Parameters()
sim_params.add("p1", 101e-3)
sim_params.add("p2",202e-4)
sim_params.add("p3",505e-5)

# We also need a spectral, a time axis and some amplitudes
spectral_axis = np.arange(600, 750, 1)
time_axis = np.arange(0, 1500, 1.5)
amplitudes = [1,0.1,7]

# Now we can simulate the data
model.fit_model().eval(simparams, **{'dataset':'dataset1', 'x': x})

AttributeError: 'NoneType' object has no attribute 'spectral_axis'

In [39]:

#fitmodel = KineticSeparableModel(model)


AttributeError: 'NoneType' object has no attribute 'spectral_axis'